# A/B Testing for ShoeFly.com

In [1]:
import requests
import pandas as pd

url = 'https://github.com/Giofabro/exercise-data-analysis/raw/main/Pandas/Excel1.xlsx'
response = requests.get(url)
open('Excel1.xlsx', 'wb').write(response.content)

ad_clicks = pd.read_excel('Excel1.xlsx')

Let's examine the first few rows of our dataframe!

In [2]:
print(ad_clicks.head())

   Unnamed: 0                               user-id utm_source           day  \
0           0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1           1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2           2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3           3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4           4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0           07:18:00                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


Piccolo errore. Rinominiamo colonna 'user-id' con 'user_id'

In [3]:
ad_clicks.rename(columns ={'user-id': 'user_id'}, inplace = True)

We evaluate which platform had more visits.

In [4]:
mayor_utm_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()

print(mayor_utm_source)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


Google is the winner! Well, let's proceed with the analysis. We want to know the percentage of people who clicked on the ads in each utm_source.

We start by grouping by utm_source and is_click and counting the number of user_ids in each of these groups. We save the answer in the clicks_by_source variable. 

In [10]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

clicks_by_source = ad_clicks\
   .groupby(['utm_source',
             'is_click'])\
   .user_id.count()\
   .reset_index()

Now let's pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.

In [11]:
clicks_pivot = clicks_by_source.pivot(
    index='utm_source',
    columns='is_click',
    values='user_id').reset_index()

We create a new column in clicks_pivot called percent_clicked that is equal to the percentage of users who clicked on the ad from each utm_source.

In [12]:
clicks_pivot['percent_clicked'] = \
   clicks_pivot[True] / \
   (clicks_pivot[True] + 
    clicks_pivot[False])

In [14]:
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977
